In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("CRJ700.csv",header=0)

In [2]:
df['features'] = df['INTERRUPTION_REASON'].str.cat(df['CORRECTIVE_ACTION'], sep='  ',na_rep=' ')

In [3]:
len(df)

25696

In [4]:
len(df1)

13454

In [3]:
import re
import nltk
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
def identify_tokens1(df):
    comment = df['features']
    comment = re.sub(r'\W',' ',comment)
    comment = re.sub(r'\s+',' ', comment)
    words = nltk.word_tokenize(comment)
    new_words = [stemmer.stem(word) for word in words]
    comment = ' '.join(new_words)
    return comment

df['features_1'] = df.apply(identify_tokens1, axis=1)

In [4]:
df1 = df.dropna(axis=0, subset=['ATA_SYMPTOM'])

In [12]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13454 entries, 0 to 25695
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unnamed: 0               13454 non-null  int64  
 1   Unnamed: 0.1             13454 non-null  int64  
 2   AI_DATE                  13454 non-null  object 
 3   AC_MODEL                 13454 non-null  object 
 4   AC_SN                    13454 non-null  int64  
 5   OPERATOR_CODE            13454 non-null  object 
 6   EVENT_RECORD_IDENTIFIER  3914 non-null   object 
 7   DIVERTED_LANDING         13454 non-null  int64  
 8   AIR_TURN_BACK            13454 non-null  object 
 9   REJECTED_TAKE_OFF        13454 non-null  object 
 10  GROUND_TURN_BACK         13454 non-null  object 
 11  ABORTED_APPROACH         13454 non-null  object 
 12  EMERGENCY_DESCENT        13454 non-null  object 
 13  SMOKE                    13454 non-null  object 
 14  EMERGENCY_LANDING     

In [5]:
from sklearn.model_selection import train_test_split
X = df1['features_1']
y = df1['ATA_SYMPTOM']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
X_test

15349    wheel chock wa left IN dure push back capt req...
24101                              pax door will not close
21802    RT wing emer door caut ato clean lube door sea...
19321    aft cargo door cau msg rtg complet by date in ...
15332    apu door open caution msg ON the ground W apu ...
                               ...                        
20351                        transpond not transmit mode C
263      birdstrik RH wing LE compli with BS inspect nd...
14600    downlin delay due TO rtg xflow pump caution ov...
7772     RT eng W N start RT eng start valv and RT eng ...
7253     dure taxi out R pack auto fail caution follow ...
Name: features_1, Length: 1346, dtype: object

In [28]:
X_1 = df1['INTERRUPTION_REASON']
y_1 = df1['ATA_SYMPTOM']
X_train1, X_test1, y_train1, y_test1 = train_test_split(X_1, y_1, test_size=0.1, random_state=42)
X_test1 

15349    WHEEL CHOCK WAS LEFT IN DURING PUSH BACK CAPT ...
24101                              PAX DOOR WILL NOT CLOSE
21802                           RT WING EMER DOOR CAUT-ATO
19321    AFT CARGO DOOR CAU MSG- RTG Complete by date i...
15332    APU DOOR OPEN CAUTION MSG ON THE GROUND W/ APU...
                               ...                        
20351                 TRANSPONDERS NOT TRANSMITTING MODE C
263                                  BIRDSTRIKE RH WING LE
14600    DOWNLINE DELAY DUE TO RTG - XFLOW PUMP CAUTION...
7772     RT ENG W/N START, RT ENG START VALVE AND RT EN...
7253     DURING TAXI OUT R PACK AUTO FAIL CAUTION FOLLO...
Name: INTERRUPTION_REASON, Length: 1346, dtype: object

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('tfidf',TfidfVectorizer(max_features=9000,stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
text_clf.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=9000, stop_words='english')),
                ('clf', CalibratedClassifierCV(base_estimator=LinearSVC()))])

In [7]:
predictions = text_clf.predict(X_test)

In [7]:
len(predictions)

1346

In [16]:
from sklearn import metrics
#metrics.confusion_matrix(y_test,predictions)

In [11]:
predictions_prob = pd.DataFrame(text_clf.predict_proba(X_test),columns=text_clf.classes_)
predictions_prob

,0.0,200.0,240.0,320.0,520.0,550.0,1120.0,1130.0,1200.0,2120.0,...,561201.0,711201.0,720001.0,722110.0,732101.0,741101.0,772101.0,783113.0,793101.0,801103.0
0,0.000901,0.109648,0.199530,0.009053,0.016455,0.000555,0.000582,0.001141,0.006655,0.000538,...,0.001811,0.000555,0.000630,0.000538,0.004180,0.000543,0.002652,0.000538,0.000538,0.000543
1,0.000456,0.012513,0.007686,0.000343,0.000547,0.000221,0.000264,0.000286,0.000054,0.000225,...,0.000248,0.000221,0.000337,0.000225,0.000421,0.000227,0.000236,0.000225,0.000225,0.000227
2,0.000679,0.010944,0.072062,0.000534,0.000425,0.000350,0.000411,0.000458,0.000057,0.000366,...,0.000423,0.000350,0.000721,0.000366,0.001140,0.000356,0.000484,0.000366,0.000366,0.000356
3,0.000750,0.040285,0.065356,0.001090,0.001328,0.000377,0.000384,0.001489,0.000593,0.000415,...,0.000515,0.000377,0.000596,0.000415,0.000644,0.000397,0.000621,0.000415,0.000415,0.000397
4,0.000452,0.004621,0.004067,0.000405,0.001294,0.000277,0.000276,0.000590,0.000337,0.000276,...,0.000357,0.000277,0.000335,0.000276,0.000254,0.000277,0.000930,0.000276,0.000276,0.000277
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,0.000641,0.015199,0.010873,0.000465,0.002000,0.000279,0.000462,0.000456,0.002518,0.000264,...,0.000411,0.000279,0.000462,0.000264,0.000657,0.000270,0.000333,0.000264,0.000264,0.000270
1342,0.000477,0.004315,0.830210,0.000435,0.002728,0.000277,0.000401,0.000392,0.000808,0.000277,...,0.002320,0.000277,0.000386,0.000277,0.002782,0.000277,0.000256,0.000277,0.000277,0.000277
1343,0.000817,0.006097,0.019394,0.000337,0.002722,0.000383,0.000556,0.000585,0.015067,0.000357,...,0.000471,0.000383,0.000496,0.000357,0.001733,0.000370,0.000431,0.000357,0.000357,0.000370
1344,0.000274,0.003091,0.023078,0.000387,0.000611,0.000265,0.000401,0.000293,0.000597,0.000265,...,0.000483,0.000265,0.000355,0.000265,0.005910,0.000265,0.004503,0.000265,0.000265,0.000265


In [9]:
top_three_perc = predictions_prob.max(axis=1)
top_three_perc 

0       0.201693
1       0.584330
2       0.610655
3       0.572409
4       0.809266
          ...   
1341    0.155257
1342    0.855659
1343    0.639181
1344    0.726088
1345    0.809153
Length: 1346, dtype: float64

In [10]:
top_three_perc1 = predictions_prob.idxmax(axis=1)
top_three_perc1 

0       324201.0
1         5211.0
2         5270.0
3         5270.0
4         4914.0
          ...   
1341      3454.0
1342       240.0
1343    284122.0
1344      8011.0
1345      2151.0
Length: 1346, dtype: float64

In [12]:
#predictions_prob.nlargest(1,0)
#predictions_prob.sort_values(by=0,axis=1,ascending=False).iloc[:,0:3]
#predictions_prob.apply(lambda x:x.sort_values(axis=1,ascending=False), axis=1)
sort_decr2_topn = lambda row, nlargest=10:sorted(pd.Series(zip(predictions_prob.columns, row)), key=lambda cv: -cv[1]) [:nlargest]

tmp = predictions_prob.apply(sort_decr2_topn, axis=1)
#tmp = np.array(tmp)
tmp

0       [(240.0, 0.19953022757502853), (3221.0, 0.1618...
1       [(5211.0, 0.6316397676525888), (5270.0, 0.1520...
2       [(5270.0, 0.5984512278832186), (240.0, 0.07206...
3       [(5270.0, 0.5035142941505558), (240.0, 0.06535...
4       [(4914.0, 0.833280476280683), (200.0, 0.004621...
                              ...                        
1341    [(3454.0, 0.7198630597685954), (2351.0, 0.0292...
1342    [(240.0, 0.8302101247524007), (200.0, 0.004315...
1343    [(284122.0, 0.6839135663184722), (240.0, 0.019...
1344    [(8011.0, 0.7945238337585562), (240.0, 0.02307...
1345    [(2151.0, 0.8267614770378471), (3621.0, 0.0052...
Length: 1346, dtype: object

In [13]:
#predictions_prob.apply(lambda s: pd.Series(s.nlargest(3)))
#np.array(tmp)
result = pd.DataFrame(list(map(np.ravel, tmp)))
result

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,240.0,0.199530,3221.0,0.161843,200.0,0.109648,324201.0,0.076440,7721.0,0.020995,520.0,0.016455,3251.0,0.014468,7321.0,0.013230,3243.0,0.011080,320.0,0.009053
1,5211.0,0.631640,5270.0,0.152039,4914.0,0.057668,200.0,0.012513,240.0,0.007686,352105.0,0.004971,2161.0,0.004507,5245.0,0.004318,2542.0,0.002579,7321.0,0.002376
2,5270.0,0.598451,240.0,0.072062,5241.0,0.069712,3012.0,0.024645,5211.0,0.011691,200.0,0.010944,4914.0,0.008774,3325.0,0.006470,3222.0,0.003752,2151.0,0.003672
3,5270.0,0.503514,240.0,0.065356,200.0,0.040285,3613.0,0.023405,3231.0,0.021708,8011.0,0.016161,252101.0,0.012139,3244.0,0.011042,2741.0,0.008935,4914.0,0.007103
4,4914.0,0.833280,200.0,0.004621,240.0,0.004067,5211.0,0.002920,4961.0,0.002787,276105.0,0.002785,2824.0,0.002447,4951.0,0.002220,3261.0,0.002097,2725.0,0.001879
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1341,3454.0,0.719863,2351.0,0.029272,200.0,0.015199,240.0,0.010873,2211.0,0.007981,2311.0,0.006190,4951.0,0.005959,2381.0,0.004912,3221.0,0.004276,8011.0,0.003964
1342,240.0,0.830210,200.0,0.004315,732101.0,0.002782,520.0,0.002728,2911.0,0.002713,8011.0,0.002602,3012.0,0.002441,561201.0,0.002320,2761.0,0.002182,3221.0,0.002106
1343,284122.0,0.683914,240.0,0.019394,1200.0,0.015067,5221.0,0.012350,2151.0,0.010667,276105.0,0.007038,5270.0,0.006920,3613.0,0.006464,200.0,0.006097,2912.0,0.005822
1344,8011.0,0.794524,240.0,0.023078,7321.0,0.010946,732101.0,0.005910,7411.0,0.005095,3613.0,0.004802,772101.0,0.004503,7931.0,0.004318,2911.0,0.003743,2611.0,0.003563


In [14]:
result.to_csv("temp.csv",index=False,quoting=3,sep=',',escapechar='\\')

In [17]:
metrics.accuracy_score(y_test,predictions)

0.8157503714710252

In [18]:
b=pd.DataFrame(text_clf.predict_proba([identify_tokens1({'INTERRUPTION_REASON':'brk ovht'})]),columns=text_clf.classes_)
b.sort_values(by=0,axis=1,ascending=False).iloc[:,0:10]

KeyError: 'features'

In [24]:
b

,0.0,200.0,240.0,320.0,520.0,550.0,1120.0,1130.0,1200.0,2120.0,...,561201.0,711201.0,720001.0,722110.0,732101.0,741101.0,772101.0,783113.0,793101.0,801103.0
0,0.000733,0.006329,0.008601,0.001059,0.00183,0.000288,0.000395,0.000335,0.002136,0.000297,...,0.000501,0.000288,0.000372,0.000297,0.001192,0.000276,0.000345,0.000297,0.000297,0.000276


In [59]:
#

In [ ]:
# res = pd.DataFrame(predictions)
# res.index = X_test.index # its important for comparison
# res.columns = ["prediction",'y_test']
# res.to_csv("prediction_results.csv")

In [21]:
#output=pd.DataFrame(data={"Real":y_test,"prediction":predictions,"Features_Preprocessed":X_test,"Features_Original":X_test1}) 
output=pd.DataFrame(data={"Real":y_test,"prediction":predictions,"Features_Preprocessed":X_test})
output.to_csv("Prediction_AI_ATA_SYM_CR700_TOP10.csv",index=False,quoting=3,sep=',',escapechar='\\')

In [13]:
X = df['features_1']
y = df['ATA_CAUSE']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [16]:
text_clf = Pipeline([('tfidf',TfidfVectorizer(max_features=4500,stop_words='english')),('clf',CalibratedClassifierCV(LinearSVC()))])
text_clf.fit(X_train,y_train)

D:\Programs\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:670: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_features=4500, stop_words='english')),
                ('clf', CalibratedClassifierCV(base_estimator=LinearSVC()))])

In [17]:
predictions = text_clf.predict(X_test)
metrics.accuracy_score(y_test,predictions)

0.621011673151751